<a href="https://colab.research.google.com/github/AstroBoy1/santander/blob/master/lstm_200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
from google.colab import drive
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn import svm
from collections import Counter
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import time
import statistics
from sklearn.model_selection import GridSearchCV
import pandas as pd
from glob import glob
import re 
np.random.seed(0) # ensure reproducibility
np.set_printoptions(suppress = True)
# Stacking
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization, GaussianNoise
from keras import callbacks
import keras.backend as K
from keras.wrappers.scikit_learn import KerasClassifier
import warnings
warnings.simplefilter("ignore")
from keras.callbacks import EarlyStopping, TensorBoard
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline
import random

Using TensorFlow backend.


In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train_fn = '/content/gdrive/My Drive/santander_data/train.csv'
valid_fn = '/content/gdrive/My Drive/santander_data/test.csv'
pred_fn = '/content/gdrive/My Drive/santander_data/submission12.csv'
train_data_df = pd.read_csv(train_fn)
test_data_df = pd.read_csv(valid_fn)
train_data_x = train_data_df.drop(columns=["ID_code", "target"]).values
train_data_y = train_data_df["target"].values
test_data_x = test_data_df.drop(columns=["ID_code"]).values
save_directory = '/content/gdrive/My Drive/santander_results/'

# LSTM

In [5]:
from keras.layers.recurrent import LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(200, 1), dropout=0.2, recurrent_dropout=0.2, return_sequences=True, return_state=True))
model.add(LSTM(16, input_shape=(200, 1), dropout=0.2, recurrent_dropout=0.2, return_sequences=True, return_state=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
num_samples = 1000
x = train_data_x[:num_samples].reshape(num_samples, 200, 1)
x.shape
y = train_data_y[:num_samples]

In [0]:
# LOGGER
class Logger(callbacks.Callback):
    def __init__(self, out_path=save_directory, patience=10, lr_patience=3, out_fn='', log_fn=''):
        self.auc = 0
        self.path = out_path
        self.fn = out_fn
        self.patience = patience
        self.lr_patience = lr_patience
        self.no_improve = 0
        self.no_improve_lr = 0

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        cv_pred = self.model.predict(self.validation_data[0], batch_size=1024)
        cv_true = self.validation_data[1]
        auc_val = roc_auc_score(cv_true, cv_pred)
        if self.auc < auc_val:
            self.no_improve = 0
            self.no_improve_lr = 0
            print("Epoch %s - best AUC: %s" % (epoch, round(auc_val, 4)))
            self.auc = auc_val
            self.model.save(self.path + self.fn, overwrite=True)
        else:
            self.no_improve += 1
            self.no_improve_lr += 1
            print("Epoch %s - current AUC: %s" % (epoch, round(auc_val, 4)))
            if self.no_improve >= self.patience:
                self.model.stop_training = True
            if self.no_improve_lr >= self.lr_patience:
                lr = float(K.get_value(self.model.optimizer.lr))
                K.set_value(self.model.optimizer.lr, 0.75*lr)
                print("Setting lr to {}".format(0.75*lr))
                self.no_improve_lr = 0
        return

In [0]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
preds = []
oof_preds = np.zeros((num_samples, 1))
c = 0
for train, valid in cv.split(x, y):
    c += 1
    logger = Logger(patience=10, out_path=save_directory, out_fn='cv_{}.h5'.format(c))
    model.fit(x[train][:], y[train][:], nb_epoch = 5, validation_data=(x[valid][:], y[valid][:]), callbacks=[logger])
    model.load_weights(save_directory + 'cv_{}.h5'.format(c))
    X_test = np.reshape(test_data_x, (200000, 200, 1))
    curr_preds = model.predict(X_test, batch_size=2048)
    oof_preds[valid] = model.predict(x[valid])
    preds.append(curr_preds)

Train on 799 samples, validate on 201 samples
Epoch 1/5
799/799 [==============================] - 10s 13ms/step - loss: 0.2852 - acc: 0.9036 - val_loss: 0.3214 - val_acc: 0.9005
Epoch 0 - best AUC: 0.6108
Epoch 2/5
704/799 [=========================>....] - ETA: 1s - loss: 0.2801 - acc: 0.9048

In [0]:
auc = roc_auc_score(y_train, oof_preds)
print("CV_AUC: {}".format(auc))

# SAVE DATA
preds = np.asarray(preds)
preds = preds.reshape((5, 200000))
preds_final = np.mean(preds.T, axis=1)
submission = pd.read_csv(save_directory + 'santander_data/sample_submission.csv')
submission['target'] = preds_final
submission.to_csv('lstm_submission.csv', index=False)